In [1]:
import pickle

In [2]:
rets = {}

In [4]:
path = "policy_dissection_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [5]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [6]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [12]:
print(averaged_ret[4])

[{'layer': 0, 'neuron_index': 18, 'freq_diff': 15.59203448010793, 'base_phase_diff': 0.9011370715415504, 'seed_num': 10}, {'layer': 0, 'neuron_index': 44, 'freq_diff': 15.690620968401896, 'base_phase_diff': 0.5743432168717014, 'seed_num': 10}, {'layer': 0, 'neuron_index': 120, 'freq_diff': 15.931802604632285, 'base_phase_diff': 0.8600755683021747, 'seed_num': 10}, {'layer': 0, 'neuron_index': 31, 'freq_diff': 17.53531829523131, 'base_phase_diff': 0.182539709086268, 'seed_num': 10}, {'layer': 0, 'neuron_index': 70, 'freq_diff': 20.396839493941734, 'base_phase_diff': 0.7599260882696834, 'seed_num': 10}, {'layer': 0, 'neuron_index': 64, 'freq_diff': 20.663566832721237, 'base_phase_diff': 0.9179641830604716, 'seed_num': 10}, {'layer': 0, 'neuron_index': 111, 'freq_diff': 22.28151084049585, 'base_phase_diff': 0.014285740832299765, 'seed_num': 10}, {'layer': 0, 'neuron_index': 8, 'freq_diff': 22.63864304912634, 'base_phase_diff': 0.001587328133887045, 'seed_num': 10}, {'layer': 0, 'neuron_in